In [ ]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python matplotlib

In [ ]:
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
#import tensorflow dpendencies
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input,Flatten
import tensorflow as tf


In [ ]:
#GPu is not overloaded
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

Setting folders

In [ ]:
os.makedirs('data/positive')
os.makedirs('data/negative')
os.makedirs('data/anchor')

In [ ]:
#uncompress Labelled Faces in the wild dataset
import tarfile

with tarfile.open('D:/facial recognition/lfw.tgz', 'r:gz') as tar:
    tar.extractall()


In [ ]:
for directory in os.listdir('D:/facial recognition/lfw'):
    for file in os.listdir(os.path.join('D:/facial recognition/lfw', directory)):
        EX_PATH = os.path.join('D:/facial recognition/lfw', directory, file)
        NEW_PATH = os.path.join('D:/facial recognition/data/negative', file)
        os.replace(EX_PATH,NEW_PATH)


capturing anchor and positive

In [ ]:
import uuid

cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame=frame[120:120+250,200:200+250, :]

    if(cv2.waitKey(1) & 0XFF == ord('a')):
        path = 'D:/facial recognition/data/anchor/' + str(uuid.uuid1()) + '.jpg'
        cv2.imwrite(path, frame)

    if(cv2.waitKey(1) & 0XFF == ord('p')):
        path = 'D:/facial recognition/data/positive/' + str(uuid.uuid1()) + '.jpg'
        cv2.imwrite(path, frame)c

    cv2.imshow('Image Collection', frame)
    if(cv2.waitKey(1) & 0XFF == ord('c')):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
#storing dataset
anc = tf.data.Dataset.list_files('D:/facial recognition/data/anchor/' + '*.jpg').take(300)
pos = tf.data.Dataset.list_files('D:/facial recognition/data/positive/' + '*.jpg').take(300)
neg = tf.data.Dataset.list_files('D:/facial recognition/data/negative/' + '*.jpg').take(300)

scalint between 0 and 1; and resizeing to 100*100*3 to optimize gradient descent

In [ ]:
def optimal(path):
    byte_img = tf.io.read_file(path)
    img = tf.io.decode_jpeg(byte_img)
    img = tf.image.resize(img, (100,100)) 
    img = img / 255 
    return img
    


Creating dataset

In [ ]:
positives = tf.data.Dataset.zip((anc, pos, tf.data.Dataset.from_tensor_slices(tf.ones(len(anc)))))
negatives = tf.data.Dataset.zip((anc, neg, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anc)))))
data=positives.concatenate(negatives)

In [ ]:
def preprocess(input_img, validation_img, label):
    return(optimal(input_img), optimal(validation_img), label)

In [ ]:
#build dataloader pipeline
data = data.map(preprocess)
data = data.cache()
data = data.shuffle(buffer_size=1024)

In [ ]:
#training partition
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)
#to not bottle the neural network
train_data = train_data.prefetch(8)


In [ ]:
#testing partition
test_data = data.skip(round(len(data)*.7))
train_data = data.take(round(len(data)*.3))
test_data = test_data.batch(16)
#to not bottle the neural network
test_data = test_data.prefetch(8)

In [ ]:
#build embedding layer

def make_embedding():
    inp =  Input(shape=(100,100, 3), name='input_image')
    
    #first block
    con1 = Conv2D(64, (10,10), activation='relu')(inp)
    mp1 = MaxPooling2D(64, (2,2), padding='same')(con1)

    #second block
    con2 = Conv2D(128, (7,7), activation='relu')(mp1)
    mp2 = MaxPooling2D(64, (2,2), padding='same')(con2)
    
    #third block
    con3 = Conv2D(128, (4,4), activation='relu')(mp2)
    mp3 = MaxPooling2D(64, (2,2), padding='same')(con3)
    
    #final block
    con4 = Conv2D(256, (4,4), activation='relu')(mp3)
    f1 = Flatten()(con4)
    d1 = Dense(4096, activation='sigmoid')(f1)
    
    return Model(inputs=[inp], outputs= [d1], name='embedding')

embedding=make_embedding()

In [ ]:
class L1Dist(Layer):
    
    def __init__(self, **kwargs):
        super().__init__()
       
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [ ]:
l1 = L1Dist()

In [ ]:
input_image = Input(name='input_img', shape=(100,100,3))
inp_embedding = embedding(input_image)

validation_image = Input(name='validation_img', shape=(100,100,3))
val_embedding = embedding(validation_image)

In [ ]:
siamese_layer = L1Dist()
distances = siamese_layer(inp_embedding, val_embedding)
classifier = Dense(1, activation='sigmoid')(distances)

In [ ]:
siamese_network = Model(inputs=[input_image, validation_image], outputs=classifier, name='SiameseNetwork')

In [ ]:
def make_siamese_model(): 
    
    # Anchor image input in the network
    input_image = Input(name='input_img', shape=(100,100,3))
    
    # Validation image in the network 
    validation_image = Input(name='validation_img', shape=(100,100,3))
    
    # Combine siamese distance components
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(embedding(input_image), embedding(validation_image))
    
    # Classification layer 
    classifier = Dense(1, activation='sigmoid')(distances)
    
    return Model(inputs=[input_image, validation_image], outputs=classifier, name='SiameseNetwork')

In [ ]:
siamese_model = make_siamese_model()


Training

In [ ]:
binary_cross_loss = tf.losses.BinaryCrossentropy()

In [ ]:
opt = tf.keras.optimizers.Adam(1e-4)

In [ ]:
@tf.function
def train_step(batch):
    with tf.GradientTape() as tape:     
        # Get anchor and positive/negative image
        X = batch[:2]
        y = batch[2]
        
        # Add batch dimension to the input data
        X = [tf.expand_dims(x, axis=0) for x in X]
        
        yhat = siamese_model(X, training=True)
        
        # Remove the extra batch dimension from the logits
        yhat = tf.squeeze(yhat, axis=0)
        
        # Reshape y to have a shape of (1,)
        y = tf.reshape(y, (1,))
        
        loss = binary_cross_loss(y, yhat)
    print(loss)
        
    grad = tape.gradient(loss, siamese_model.trainable_variables)
    
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))
    
    return loss



In [ ]:
def train(data, EPOCHS):
    # Loop through epochs
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))
        
        # Loop through each batch
        for idx, batch in enumerate(data):
            # Run train step here
            train_step(batch)
            progbar.update(idx+1)

In [ ]:
EPOCHS = 8

In [ ]:
train(train_data, EPOCHS)

In [ ]:
from tensorflow.keras.metrics import Precision, Recall

In [ ]:
test_input, test_val, y_true = test_data.as_numpy_iterator().next()

In [ ]:
y_hat = siamese_model.predict([test_input, test_val])

In [ ]:
siamese_model.save('siamesemodel.h5')

In [ ]:
model = tf.keras.models.load_model('siamesemodel.h5', 
                                   custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

In [ ]:
model.predict([test_input, test_val])

In [ ]:
m = Recall()

# Calculating the recall value 
m.update_state(y_true, y_hat)

# Return Recall Result
m.result().numpy()

In [ ]:
m = Precision()

# Calculating the recall value 
m.update_state(y_true, y_hat)

# Return Recall Result
m.result().numpy()

In [ ]:
plt.figure(figsize=(10,8))

# Set first subplot
plt.subplot(1,2,1)
plt.imshow(test_input[0])

# Set second subplot
plt.subplot(1,2,2)
plt.imshow(test_val[0])

# Renders cleanly
plt.show()

In [ ]:
siamese_model.save('siamesemodel.h5')

In [ ]:
model = tf.keras.models.load_model('siamesemodel.h5', 
                                   custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

In [ ]:
model.predict([test_input, test_val])

In [ ]:
model.summary()

In [ ]:
def verify(model, detection_threshold, verification_threshold):
    # Build results array
    results = []
    for image in os.listdir('D:/facial recognition/application_data/verification_images'):
        input_img = optimal(os.path.join('D:/facial recognition/application_data/input_image/input_image.jpg'))
        validation_img = optimal(os.path.join('D:/facial recognition/application_data/verification_images', image))
        
        # Make Predictions 
        result = model.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        results.append(result)
    
    # Detection Threshold: Metric above which a prediciton is considered positive 
    detection = np.sum(np.array(results) > detection_threshold)
    
    # Verification Threshold: Proportion of positive predictions / total positive samples 
    verification = detection / len(os.listdir('D:/facial recognition/application_data/verification_images')) 
    verified = verification > verification_threshold
    
    return results, verified

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[120:120+250,200:200+250, :]
    
    cv2.imshow('Verification', frame)
    
    # Verification trigger
    if cv2.waitKey(10) & 0xFF == ord('v'):
        # Save input image to application_data/input_image folder 
        # cv2.imwrite(os.path.join('D:/facial recognition/application_data/input_image/input_image.jpg'), frame)
        # Run verification
        results, verified = verify(model, 0.9, 0.9)
        print(verified)
    
    if cv2.waitKey(10) & 0xFF == ord('c'):
        break
cap.release()
cv2.destroyAllWindows()